In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2  
from tqdm import tqdm
from random import randint
import json
import matplotlib.image as mpimg
from matplotlib import rcParams
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Using TensorFlow backend.


In [2]:
def show_image(images,name,row,col,size1=15,size2=15):
    n = len(images)
    plt.subplots(figsize=(15, 15))

    for i in range(n):
        plt.subplot(row,col,i+1)
        plt.title(name[i])
        plt.imshow(images[i], cmap='gray')
        plt.axis('off')   
    plt.show()

In [3]:
train_path = "data/train"
test_path = "data/test"

In [4]:
train = os.listdir(train_path)
test = os.listdir(test_path)

In [8]:
train_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(train_path, image)), cv2.COLOR_BGR2RGB),(200,200))  for image in train ]

In [9]:
test_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(test_path, image)), cv2.COLOR_BGR2RGB),(200,200))  for image in test ]

In [10]:
label = pd.read_csv("data/train_features.csv")
label[["x","y"]]

,x,y
0,-9.380678,3.58272
1,-9.380678,3.58272
2,-9.380678,3.58272
3,-9.380678,3.58272
4,-9.380678,3.58272
...,...,...
7495,-4.680678,35.18272
7496,-4.680678,35.18272
7497,-4.680678,35.18272
7498,-4.680678,35.18272


In [11]:
y_label = []
for index, row in label.iterrows():
    y_label.append((row["x"],row["y"]))
y_label = np.array(y_label)
train_image = np.array(train_image)

In [20]:
test_image = np.array(test_image)

In [34]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [13]:
pre_model = ResNet50(weights='imagenet',input_shape=[200,200,3],include_top=False)

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model

In [17]:
for layer in pre_model.layers:
    layer.trainable = False

conv1 = layers.Conv2D(filters = 32,kernel_size = (1,1),padding = "same",activation = "relu")(pre_model.layers[-1].output)
conv2 = layers.Conv2D(filters = 64,kernel_size = (3,3),padding = "same",activation = "relu")(conv1)
conv3 = layers.Conv2D(filters = 64,kernel_size = (3,3),padding = "same",activation = "relu")(conv2)
mp = layers.MaxPooling2D(pool_size=(2,2), strides=2,padding="same")(conv3)
flat = layers.Flatten()(mp)
dropout = layers.Dropout(0.5)(flat)
dense1 = layers.Dense(1024,activation="relu")(dropout)
dense2 = layers.Dense(2,activation="linear")(dense1)

model = Model(inputs = pre_model.inputs,outputs = dense2)


In [18]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
            loss=tf.keras.losses.MeanAbsoluteError())

In [36]:
model.fit(train_image, label[["x","y"]], epochs=40, batch_size=128)

Epoch 1/40
59/59 [==============================] - 19s 326ms/step - loss: 7.1959
Epoch 2/40
59/59 [==============================] - 19s 328ms/step - loss: 6.6978
Epoch 3/40
59/59 [==============================] - 19s 329ms/step - loss: 6.6887
Epoch 4/40
59/59 [==============================] - 19s 329ms/step - loss: 6.5046
Epoch 5/40
59/59 [==============================] - 19s 329ms/step - loss: 6.7688
Epoch 6/40
59/59 [==============================] - 19s 329ms/step - loss: 6.2619
Epoch 7/40
59/59 [==============================] - 20s 331ms/step - loss: 6.4266
Epoch 8/40
59/59 [==============================] - 20s 331ms/step - loss: 6.1108
Epoch 9/40
59/59 [==============================] - 20s 332ms/step - loss: 5.9711
Epoch 10/40
59/59 [==============================] - 19s 327ms/step - loss: 5.7627
Epoch 11/40
59/59 [==============================] - 19s 329ms/step - loss: 5.8795
Epoch 12/40
59/59 [==============================] - 19s 329ms/step - loss: 5.6167
Epoch 13/40
5

In [38]:
result = model.predict(test_image)


In [39]:
result[0]

array([ 23.909966, -80.70059 ], dtype=float32)

In [40]:
x = []
y = []
for i in result:
   x.append(i[0])
   y.append(i[1]) 

In [41]:
test_result = pd.read_csv("data/imagenames.csv")

In [42]:
test_result["x"] = x
test_result["y"] = y

In [ ]:
test_result.to_csv("result/CNN_baseline_40.csv",index=False)